<a href="https://colab.research.google.com/github/MehrdadJannesar/CNN_AdaBoost/blob/master/Copy_of_Copy_of_Untitled12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Load dataset
from keras.datasets import cifar10
import keras
(train_images, train_labels),(test_images, test_labels) = cifar10.load_data()

Using TensorFlow backend.


In [2]:
#Dataset's information
print("train_images dimention: " , train_images.ndim)
print("train_images shape: ", train_images.shape)
print("train_image type: ", train_images.dtype)

train_images dimention:  4
train_images shape:  (50000, 32, 32, 3)
train_image type:  uint8


In [3]:
cifar10.load_data()

((array([[[[ 59,  62,  63],
           [ 43,  46,  45],
           [ 50,  48,  43],
           ...,
           [158, 132, 108],
           [152, 125, 102],
           [148, 124, 103]],
  
          [[ 16,  20,  20],
           [  0,   0,   0],
           [ 18,   8,   0],
           ...,
           [123,  88,  55],
           [119,  83,  50],
           [122,  87,  57]],
  
          [[ 25,  24,  21],
           [ 16,   7,   0],
           [ 49,  27,   8],
           ...,
           [118,  84,  50],
           [120,  84,  50],
           [109,  73,  42]],
  
          ...,
  
          [[208, 170,  96],
           [201, 153,  34],
           [198, 161,  26],
           ...,
           [160, 133,  70],
           [ 56,  31,   7],
           [ 53,  34,  20]],
  
          [[180, 139,  96],
           [173, 123,  42],
           [186, 144,  30],
           ...,
           [184, 148,  94],
           [ 97,  62,  34],
           [ 83,  53,  34]],
  
          [[177, 144, 116],
           [16

In [0]:
#Optimazation and categorical dataset
X_train = train_images.astype('float32')
X_test = test_images.astype('float32')


X_train /= 255
X_test /= 255


N_classes = 10
cifar_classes = ["airplane", "automobile", "bird", "cat", "deer", 
                   "dog", "frog", "horse", "ship", "truck"]

Y_train = keras.utils.to_categorical(train_labels,N_classes)
Y_test = keras.utils.to_categorical(test_labels,N_classes)

In [0]:
from keras.models import Model
from keras.layers import Dense,Input,Flatten,Dropout
from keras.applications.resnet50 import ResNet50

In [6]:
#Connect to google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
#Create model
input_layer = Input((32, 32, 3))
resnet = ResNet50(include_top = False, pooling = 'avg'
                  , weights = '/content/gdrive/My Drive/Data/Data/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')(input_layer)

dense = Dense(N_classes, activation = 'softmax')(resnet)

myModel = Model(input_layer, dense)


In [8]:
#Summary model
myModel.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 10)                20490     
Total params: 23,608,202
Trainable params: 23,555,082
Non-trainable params: 53,120
_________________________________________________________________


In [9]:
#Initial optimizer
import keras
sgd = keras.optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
myModel.compile(optimizer = sgd, loss = keras.losses.categorical_crossentropy, metrics = ['accuracy'])

In [0]:
#CallBacks
#Save model

myModel.save('/content/gdrive/My Drive/Data/Data/model{epoch}_func_api.h5')
myModel.save_weights('/content/gdrive/My Drive/Data/Data/model_weights_func_api.h5')

from keras import callbacks


model_checkpoint = callbacks.ModelCheckpoint('/content/gdrive/My Drive/Data/Data/model_func_api.h5')
model_log = callbacks.CSVLogger('/content/gdrive/My Drive/Data/Data/model_func_api.log')
model_tensor = callbacks.TensorBoard('/content/gdrive/My Drive/Data/Data/Tensorboard/run2/_func_api ')
callbacks= [model_checkpoint, model_log, model_tensor]

In [0]:
#Load model

# from keras.models import load_model

# model_load = load_model('/content/gdrive/My Drive/Data/Data/model{epoch}.h5')


In [11]:
#Learning network
import datetime

start = datetime.datetime.now()

network_history = myModel.fit(X_train, Y_train, batch_size=128, epochs=10 ,validation_split=0.2
                             ,callbacks=callbacks)

end = datetime.datetime.now()

print('time compile is : ', str(end - start))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 40000 samples, validate on 10000 samples


Epoch 1/10
40000/40000 [==============================] - 86s 2ms/step - loss: 1.9973 - acc: 0.3343 - val_loss: 2.9316 - val_acc: 0.2474

Epoch 2/10
40000/40000 [==============================] - 75s 2ms/step - loss: 1.3863 - acc: 0.4981 - val_loss: 1.4277 - val_acc: 0.4880
Epoch 3/10
40000/40000 [==============================] - 75s 2ms/step - loss: 1.0643 - acc: 0.6195 - val_loss: 1.4681 - val_acc: 0.5248
Epoch 4/10
40000/40000 [==============================] - 75s 2ms/step - loss: 0.8602 - acc: 0.6963 - val_loss: 0.9380 - val_acc: 0.6690
Epoch 5/10
40000/40000 [==============================] - 75s 2ms/step - loss: 0.7089 - acc: 0.7519 - val_loss: 1.0583 - val_acc: 0.6568
Epoch 6/10
40000/40000 [==============================] - 75s 2ms/step - loss: 0.5917 - acc: 0.7910 - val_loss: 1.0103 - val_acc: 0.6700
Epoch 7/10
40000/40000 [=====

In [12]:
input_ress = myModel.get_layer(index=1).inputs
output_ress = myModel.get_layer(index=1).outputs
ress_model = Model(inputs=input_ress,outputs=output_ress)

ress_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
____________________________________________________________________________________________

In [13]:
#Predict network
test_loss, test_acc = myModel.evaluate(X_test, Y_test)
test_prediction = myModel.predict(X_test)


print("test_loss: ",test_loss)
print("test_acc: ",test_acc)
print("test_prediction", test_prediction)

10000/10000 [==============================] - 12s 1ms/step
test_loss:  1.0961406462669372
test_acc:  0.7061
test_prediction [[1.1082089e-03 7.7668135e-04 3.2990326e-03 ... 4.4688574e-04
  3.7831333e-03 1.3512875e-03]
 [1.2541873e-02 9.3382519e-01 1.1960048e-04 ... 3.6545061e-05
  4.8106480e-02 5.0072824e-03]
 [2.1848740e-01 1.7156130e-02 1.6758339e-03 ... 4.1350117e-03
  2.9914302e-01 4.5148599e-01]
 ...
 [7.5710979e-08 6.1049676e-09 1.0765114e-05 ... 3.2242886e-07
  1.5480776e-08 3.5759857e-08]
 [2.0996884e-01 6.9524854e-01 2.0314302e-02 ... 2.4579156e-02
  8.3771840e-05 2.7759133e-02]
 [6.1749682e-07 7.1952528e-09 1.0717707e-06 ... 9.9808621e-01
  1.2341106e-09 2.0005932e-09]]


In [94]:
import numpy as np
print("\n=============================predict_test_uniq_1================================\n\n\n")
#Resnet_to_add_13
input_ress = myModel.get_layer(index=1).inputs

output_ress = myModel.get_layer('resnet50').get_layer('add_13').output

adaboostLayer_1 = keras.layers.GlobalAveragePooling2D()(output_ress)

adaboostLayer_1 = Dropout(0.7)(adaboostLayer_1)

dens_adaboostLayer_1 = Dense(N_classes, activation= 'softmax')(adaboostLayer_1)

adaboostLayer_1 = Model(inputs=input_ress,outputs=dens_adaboostLayer_1)

sgd = keras.optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)

adaboostLayer_1.compile(optimizer = sgd, loss = keras.losses.categorical_crossentropy, metrics = ['accuracy'])

adaboostLayer_1_his = adaboostLayer_1.fit(X_train, Y_train, batch_size=128, epochs=5 ,validation_split=0.2)

predict_adaLayer_1 = adaboostLayer_1.predict(X_test)

predict_test_uniq_1 = np.argmax(predict_adaLayer_1, axis=1)

print("\n========================================================================================\n")
print("predict_test_uniq_1 ===============>>> ",predict_test_uniq_1)
print("\n========================================================================================\n\n\n")
print("\n=============================predict_test_uniq_2================================\n\n\n")

#Resnet_to_add_14
input_ress = myModel.get_layer(index=1).inputs

output_ress = myModel.get_layer('resnet50').get_layer('add_14').output

adaboostLayer_2 = keras.layers.GlobalAveragePooling2D()(output_ress)

adaboostLayer_2 = Dropout(0.7)(adaboostLayer_2)

dens_adaboostLayer_2 = Dense(N_classes, activation= 'softmax')(adaboostLayer_2)

adaboostLayer_2 = Model(inputs=input_ress,outputs=dens_adaboostLayer_2)

sgd = keras.optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)

adaboostLayer_2.compile(optimizer = sgd, loss = keras.losses.categorical_crossentropy, metrics = ['accuracy'])

adaboostLayer_2_his = adaboostLayer_2.fit(X_train, Y_train, batch_size=128, epochs=5 ,validation_split=0.2)

predict_adaLayer_2 = adaboostLayer_2.predict(X_test)

predict_test_uniq_2 = np.argmax(predict_adaLayer_2, axis=1)



print("\n========================================================================================\n")
print("predict_test_uniq_2 ===============>>> ",predict_test_uniq_2)
print("\n========================================================================================\n\n\n")
print("\n=============================predict_test_uniq_3================================\n\n\n")

#Resnet_to_add_15
input_ress = myModel.get_layer(index=1).inputs

output_ress = myModel.get_layer('resnet50').get_layer('add_15').output

adaboostLayer_3 = keras.layers.GlobalAveragePooling2D()(output_ress)

adaboostLayer_2 = Dropout(0.7)(adaboostLayer_3)

dens_adaboostLayer_3 = Dense(N_classes, activation= 'softmax')(adaboostLayer_3)

adaboostLayer_3 = Model(inputs=input_ress,outputs=dens_adaboostLayer_3)

sgd = keras.optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)

adaboostLayer_3.compile(optimizer = sgd, loss = keras.losses.categorical_crossentropy, metrics = ['accuracy'])

adaboostLayer_3_his = adaboostLayer_3.fit(X_train, Y_train, batch_size=128, epochs=5 ,validation_split=0.2)

predict_adaLayer_3 = adaboostLayer_3.predict(X_test)

predict_test_uniq_3 = np.argmax(predict_adaLayer_3, axis=1)





print("\n========================================================================================\n")
print("predict_test_uniq_3 ===============>>> ",predict_test_uniq_3)
print("\n========================================================================================\n\n\n")
print("\n=============================predict_test_uniq_4================================\n\n\n")

#Resnet_to_add_16
input_ress = myModel.get_layer(index=1).inputs

output_ress = myModel.get_layer('resnet50').get_layer('add_16').output

adaboostLayer_4 = keras.layers.GlobalAveragePooling2D()(output_ress)

adaboostLayer_4 = Dropout(0.7)(adaboostLayer_4)

dens_adaboostLayer_4 = Dense(N_classes, activation= 'softmax')(adaboostLayer_4)

adaboostLayer_4 = Model(inputs=input_ress,outputs=dens_adaboostLayer_4)

sgd = keras.optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)

adaboostLayer_4.compile(optimizer = sgd, loss = keras.losses.categorical_crossentropy, metrics = ['accuracy'])

adaboostLayer_4_his = adaboostLayer_4.fit(X_train, Y_train, batch_size=128, epochs=5 ,validation_split=0.2)

predict_adaLayer_4 = adaboostLayer_4.predict(X_test)

predict_test_uniq_4 = np.argmax(predict_adaLayer_4, axis=1)

print("\n========================================================================================\n")
print("predict_test_uniq_4 ===============>>> ",predict_test_uniq_4)
print("\n========================================================================================\n\n\n")
print("\n=============================Done================================\n\n\n")


=============================predict_test_uniq_1================================



Train on 40000 samples, validate on 10000 samples
Epoch 1/5
40000/40000 [==============================] - 69s 2ms/step - loss: 0.2633 - acc: 0.9142 - val_loss: 1.1942 - val_acc: 0.7319
Epoch 2/5
40000/40000 [==============================] - 54s 1ms/step - loss: 0.1736 - acc: 0.9422 - val_loss: 1.5251 - val_acc: 0.6790
Epoch 3/5
40000/40000 [==============================] - 54s 1ms/step - loss: 0.1339 - acc: 0.9555 - val_loss: 1.2669 - val_acc: 0.7345
Epoch 4/5
40000/40000 [==============================] - 54s 1ms/step - loss: 0.1137 - acc: 0.9627 - val_loss: 1.3164 - val_acc: 0.7351
Epoch 5/5
40000/40000 [==============================] - 54s 1ms/step - loss: 0.0898 - acc: 0.9708 - val_loss: 1.2966 - val_acc: 0.7388


predict_test_uniq_1 ===============>>>  [8 1 8 ... 5 1 7]





=============================predict_test_uniq_2================================



Train on 40000 samples, validate on 

In [97]:
#Ensembel method (AdaBoost)
from sklearn.ensemble import AdaBoostClassifier

ada_boost_predict_1 = AdaBoostClassifier()
ada_boost_predict_2 = AdaBoostClassifier()
ada_boost_predict_3 = AdaBoostClassifier()
ada_boost_predict_4 = AdaBoostClassifier()

ada_boost_predict_1.fit(predict_test_uniq_1.reshape(-1, 1), test_labels)
ada_boost_predict_2.fit(predict_test_uniq_2.reshape(-1, 1), test_labels)
ada_boost_predict_3.fit(predict_test_uniq_3.reshape(-1, 1), test_labels)
ada_boost_predict_4.fit(predict_test_uniq_4.reshape(-1, 1), test_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=None)

In [0]:
#Feature Adaboost
feature_predict_1 = ada_boost_predict_1.feature_importances_  
feature_predict_2 = ada_boost_predict_2.feature_importances_  
feature_predict_3 = ada_boost_predict_3.feature_importances_  
feature_predict_4 = ada_boost_predict_4.feature_importances_  

In [0]:
#Predict Adaboost
ada_predict_1 = ada_boost_predict_1.predict(predict_test_uniq_1.reshape(-1, 1))
ada_predict_2 = ada_boost_predict_2.predict(predict_test_uniq_2.reshape(-1, 1))
ada_predict_3 = ada_boost_predict_3.predict(predict_test_uniq_3.reshape(-1, 1))
ada_predict_4 = ada_boost_predict_4.predict(predict_test_uniq_4.reshape(-1, 1))

In [102]:
#Score Adaboost
ada_predict_1_score = ada_boost_predict_1.score(predict_test_uniq_1.reshape(-1, 1), test_labels)
ada_predict_2_score = ada_boost_predict_2.score(predict_test_uniq_2.reshape(-1, 1), test_labels)
ada_predict_3_score = ada_boost_predict_3.score(predict_test_uniq_3.reshape(-1, 1), test_labels)
ada_predict_4_score = ada_boost_predict_4.score(predict_test_uniq_4.reshape(-1, 1), test_labels)

print("ada_predict_1_score: ",ada_predict_1_score)
print("ada_predict_2_score: ",ada_predict_2_score)
print("ada_predict_3_score: ",ada_predict_3_score)
print("ada_predict_4_score: ",ada_predict_4_score)

ada_predict_1_score:  0.7396
ada_predict_2_score:  0.2132
ada_predict_3_score:  0.3232
ada_predict_4_score:  0.323


In [0]:
#Multi-class AdaBoosted

from sklearn.metrics import accuracy_score

real_test_errors = []
discrete_test_errors = []
#predict_test_uniq_1, test_labels
for real_test_predict, discrete_train_predict in zip(
        ada_boost_predict_1.staged_predict(predict_test_uniq_1.reshape(-1, 1)), ada_boost_predict_2.staged_predict(predict_test_uniq_2.reshape(-1, 1))):
    real_test_errors.append(
        1. - accuracy_score(real_test_predict, test_labels))
    discrete_test_errors.append(
        1. - accuracy_score(discrete_train_predict, test_labels))

    
    
    
n_trees_discrete = len(ada_boost_predict_2)
n_trees_real = len(ada_boost_predict_1)

# Boosting might terminate early, but the following arrays are always
# n_estimators long. We crop them to the actual number of trees here:
discrete_estimator_errors = ada_boost_predict_2.estimator_errors_[:n_trees_discrete]
real_estimator_errors = ada_boost_predict_1.estimator_errors_[:n_trees_real]
discrete_estimator_weights = ada_boost_predict_2.estimator_weights_[:n_trees_discrete]

In [0]:
real_test_errors_2 = []
discrete_test_errors_2 = []
#predict_test_uniq_1, test_labels
for real_test_predict, discrete_train_predict in zip(
        ada_boost_predict_3.staged_predict(predict_test_uniq_3.reshape(-1, 1)), ada_boost_predict_4.staged_predict(predict_test_uniq_4.reshape(-1, 1))):
    real_test_errors_2.append(
        1. - accuracy_score(real_test_predict, test_labels))
    discrete_test_errors_2.append(
        1. - accuracy_score(discrete_train_predict, test_labels))

    
    
    
n_trees_discrete_2 = len(ada_boost_predict_4)
n_trees_real_2 = len(ada_boost_predict_3)

# Boosting might terminate early, but the following arrays are always
# n_estimators long. We crop them to the actual number of trees here:
discrete_estimator_errors_2 = ada_boost_predict_4.estimator_errors_[:n_trees_discrete_2]
real_estimator_errors_2 = ada_boost_predict_3.estimator_errors_[:n_trees_real_2]
discrete_estimator_weights_2 = ada_boost_predict_4.estimator_weights_[:n_trees_discrete_2]